In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from sklearn import linear_model
import scipy.stats as st
from pandas.core import datetools
%matplotlib inline

C:\Users\Yiheng\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  


In [2]:
month = {'JAN': 1, 'FEB':2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT':10, 'NOV': 11, 'DEC': 12}

In [3]:
new_company_list = ['ASGN', 'AVA', 'B', 'BCPC', 'CACI', 'CBU', 'CHE', 'COLB', 'FCFS', 'FFIN', 'FIVE', 'GBCI', 'GDOT',
                    'HAE', 'HI', 'IART', 'JBT', 'KFY', 'KS', 'NEOG', 'NSP', 'PDCE', 'PSB', 'QLYS', 'SIGI', 'SKYW',
                     'STMP', 'TREX', 'VAC', 'WWE']

In [4]:
len(new_company_list)

30

In [5]:
# Import high frequency data
def import_data(path, company_list):
    path = path               #r'D:\data_raw' # Please put all .csv documents into one folder
    company_list = company_list# Input company name as a list
    company_data = {} # Store company data as a dictionary: {company_name1: data1, company_name2: data2, ...}
    for company in company_list:
        frame = pd.DataFrame()
        list_ = []
        allFiles = glob.glob(path + '/' +company +'_' + '*.csv')
        allFiles.reverse()
        #print (allFiles)
        for file_ in allFiles:
            df = pd.read_csv(file_, index_col=None, header=0, usecols=[0,1], skiprows=[1,])
            list_.append(df)
        frame = pd.concat(list_)
        company_data[company] = frame
        company_data[company].index = pd.RangeIndex(len(company_data[company].index))
    return company_data

## 导入数据，要改下面路径！

In [6]:
new_company_data = import_data(r'C:/Users/Yiheng/ORIE-5370-project/New_Data', new_company_list)

In [7]:
def print_shape(dict_df, company_list):
    for com in company_list:
        print (com, dict_df[com].shape)

In [8]:
print_shape(new_company_data, new_company_list)

('ASGN', (97217, 2))
('AVA', (97975, 2))
('B', (95697, 2))
('BCPC', (82979, 2))
('CACI', (91975, 2))
('CBU', (91644, 2))
('CHE', (85200, 2))
('COLB', (94750, 2))
('FCFS', (92945, 2))
('FFIN', (89271, 2))
('FIVE', (98667, 2))
('GBCI', (97197, 2))
('GDOT', (97619, 2))
('HAE', (96836, 2))
('HI', (94420, 2))
('IART', (93950, 2))
('JBT', (88779, 2))
('KFY', (97263, 2))
('KS', (98597, 2))
('NEOG', (86160, 2))
('NSP', (88395, 2))
('PDCE', (98858, 2))
('PSB', (79732, 2))
('QLYS', (93020, 2))
('SIGI', (90061, 2))
('SKYW', (97536, 2))
('STMP', (92793, 2))
('TREX', (95969, 2))
('VAC', (94561, 2))
('WWE', (96903, 2))


In [9]:
# cluster by day
company_data=new_company_data
date_list=[]
daily={}
time_list=company_data['AVA']['Time Interval'][1:79]#.drop([1],axis=0,inplacd=True)#因后面计算收益率，去掉第一个时间标
#time_list=time_list.reset_index(drop=True)
for com in ['AVA']:    
    date_idx_s = np.array(company_data[com].loc[pd.isnull(company_data[com].Close), ].index) # record starting ind of partition b/t days
    date_idx_e = np.append(date_idx_s[1:], company_data[com].shape[0])# record end ind of partition b/t days
    for i, j in zip(date_idx_s, date_idx_e):
        date_list.append(company_data[com]['Time Interval'][i][:9])
        daily[date_list[-1]]=pd.DataFrame(columns = ['Time Interval'].append(new_company_list))

date_pt=0
clean_company_list=['AVA','FIVE','GDOT','KS','PDCE']
for d in date_list:    
    daily[d]['Time Interval']=time_list#在每天下面建立时间轴,因后面计算收益率，去掉第一个时间标
    for com in clean_company_list:#new_company_list:
        date_idx_s = np.array(company_data[com].loc[pd.isnull(company_data[com].Close), ].index) # record starting ind of partition b/t days
        date_idx_e = np.append(date_idx_s[1:], company_data[com].shape[0])# record end ind of partition b/t days
        

        if date_idx_e[date_pt]-date_idx_s[date_pt]==79:
            # 计算5分钟收益率：为了绕过df 的index的限制，分子分母分开处理
            tmp1=np.array(company_data[com]['Close'][date_idx_s[date_pt]+1:date_idx_e[date_pt]].diff())
            tmp2=np.append(np.nan,np.array(company_data[com]['Close'][date_idx_s[date_pt]+1:date_idx_e[date_pt]])[:-1])            
            daily[d][com]=tmp1/tmp2
        else:
            tmp=[]
            tmp1=None
            tmp2=None
            j=1
            for i in range(1,79):#range(date_idx_s[date_pt]+1,date_idx_e[date_pt]):
                if company_data[com]['Time Interval'][j]==daily[d]['Time Interval'][i]:
                    tmp.append(company_data[com]['Close'][j])
                    j=j+1
                else:
                    tmp.append(np.nan)
            tmp1=np.append(np.nan,np.diff(tmp))
            tmp2=np.append(np.nan,tmp[:-1])
            daily[d][com]=tmp1/tmp2#company_data[com]['Close'][date_idx_s[date_pt]+1:date_idx_e[date_pt]].diff()/company_data[com]['Close'][date_idx_s[date_pt]+1:date_idx_e[date_pt]]
 

In [11]:
company_data['AVA'][1:73]
daily['05MAR2013']

,Time Interval,ASGN,AVA,FIVE,GBCI,GDOT,KFY,KS,PDCE,SKYW,WWE
1,09:30 - 09:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09:35 - 09:40,0.006637,0.003434,-0.003738,0.003413,-0.009168,0.004444,0.004362,-0.000316,-0.003388,0.000000
3,09:40 - 09:45,0.000000,0.004183,0.001452,0.000283,-0.007931,0.000000,-0.000362,-0.005064,0.001360,-0.001156
4,09:45 - 09:50,-0.003516,-0.001893,-0.005320,0.000850,0.003997,-0.002212,-0.001448,0.001697,-0.002709,0.001157
5,09:50 - 09:55,0.001323,0.001138,-0.000484,-0.001699,-0.001327,0.004989,0.002901,0.007622,0.001355,NaN
6,09:55 - 10:00,0.007930,0.002274,-0.001216,0.000000,0.001661,-0.002206,0.005785,0.004413,0.002039,NaN
7,10:00 - 10:05,0.004371,-0.001512,0.001947,0.001134,-0.001658,-0.002211,-0.001078,-0.005230,-0.001357,-0.004630
8,10:05 - 10:10,-0.006092,-0.001136,-0.000364,0.000567,0.000664,0.005263,-0.000720,-0.002734,0.000000,-0.001163
9,10:10 - 10:15,0.004378,-0.001516,0.000122,0.002265,-0.001992,-0.000827,-0.001440,0.002320,0.000000,0.006985
10,10:15 - 10:20,-0.000436,0.000759,-0.000002,-0.002254,0.004657,-0.003861,-0.000721,-0.003997,-0.003261,0.000578


In [79]:
daily_clean={}
for d in date_list: 
    #daily_clean[d]=daily[d].drop(['PSB','BCPC','CHE','NEOG','NSP','JBT','FFIN','SIGI'], axis=1)
    daily_clean[d]=daily[d].dropna(axis=0, how='any')
    print daily_clean[d].shape

(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)


(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)
(77, 6)


In [80]:
daily_clean['05MAR2013']

,Time Interval,AVA,FIVE,GDOT,KS,PDCE
2,09:35 - 09:40,0.003434,-0.003738,-0.009168,0.004362,-0.000316
3,09:40 - 09:45,0.004183,0.001452,-0.007931,-0.000362,-0.005064
4,09:45 - 09:50,-0.001893,-0.005320,0.003997,-0.001448,0.001697
5,09:50 - 09:55,0.001138,-0.000484,-0.001327,0.002901,0.007622
6,09:55 - 10:00,0.002274,-0.001216,0.001661,0.005785,0.004413
7,10:00 - 10:05,-0.001512,0.001947,-0.001658,-0.001078,-0.005230
8,10:05 - 10:10,-0.001136,-0.000364,0.000664,-0.000720,-0.002734
9,10:10 - 10:15,-0.001516,0.000122,-0.001992,-0.001440,0.002320
10,10:15 - 10:20,0.000759,-0.000002,0.004657,-0.000721,-0.003997
11,10:20 - 10:25,-0.000569,-0.000727,-0.003974,-0.000361,-0.000845


In [116]:
st.skew(daily_clean['05MAR2013']['FIVE'])
#np.var(daily_clean['05MAR2013']['AVA'],ddof=1)

-0.9639518328593133

In [128]:
high_moments={}
for d in date_list:#:['05MAR2013']
    tmp=np.mat(daily_clean[d][clean_company_list])
    covar=np.cov(tmp,rowvar=False) # each row is an obs   
    #high_moments[d]=high_moments[d].append({'covar':covar})#pd.DataFrame(columns = ['covar','coskew','cokurt'])
    
    ## coskewness
#     % Computation of M3, the co-skewness matrix
    for i in range(tmp.shape[1]):
        S=np.zeros((tmp.shape[1],tmp.shape[1]))
        for j in range(tmp.shape[1]):
            for k in range(tmp.shape[1]):
                u=0
                for t in range(tmp.shape[0]):
                    u=u+((tmp[t,i]-np.mean(tmp[:,i]))*(tmp[t,j]-np.mean(tmp[:,j]))*(tmp[t,k]-np.mean(tmp[:,k])))
                S[j,k]=(u+0.0)/(tmp.shape[0]-1)
        if i==0:
            M3=S
        else:
            M3=np.hstack((M3,S))
    
    

    ## cokurtosis
    # 运行时间太长了，先不计算 协峰度矩阵
    
#     for i in range(tmp.shape[1]):
#         for j in range(tmp.shape[1]):
#             S=np.zeros((tmp.shape[1],tmp.shape[1]))
#             for k in range(tmp.shape[1]):
#                 for l in range(tmp.shape[1]):
#                     u=0;
#                     for t in range(tmp.shape[0]): 
#                         u=u+((tmp[t,i]-np.mean(tmp[:,i]))*(tmp[t,j]-np.mean(tmp[:,j]))*(tmp[t,k]-np.mean(tmp[:,k]))*(tmp[t,l]-np.mean(tmp[:,l])))
#                     S[k,l]=(u+0.0)/(tmp.shape[0]-1)                
#             if i==0 and j==0:
#                 M4=S
#             else:
#                 M4=np.hstack((M4,S))
#     high_moments[d]={'covar':covar,'coskew':M3,'cokurt':M4}
    print d

05MAR2013
06MAR2013
07MAR2013
08MAR2013
11MAR2013
12MAR2013
13MAR2013
14MAR2013
15MAR2013
18MAR2013
19MAR2013
20MAR2013
21MAR2013
22MAR2013
25MAR2013
26MAR2013
27MAR2013
28MAR2013
01APR2013
02APR2013
03APR2013
04APR2013
05APR2013
08APR2013
09APR2013
10APR2013
11APR2013
12APR2013
15APR2013
16APR2013
17APR2013
18APR2013
19APR2013
22APR2013
23APR2013
24APR2013
25APR2013
26APR2013
29APR2013
30APR2013
01MAY2013
02MAY2013
03MAY2013
06MAY2013
07MAY2013
08MAY2013
09MAY2013
10MAY2013
13MAY2013
14MAY2013
15MAY2013
16MAY2013
17MAY2013
20MAY2013
21MAY2013
22MAY2013
23MAY2013
24MAY2013
28MAY2013
29MAY2013
30MAY2013
31MAY2013
03JUN2013
04JUN2013
05JUN2013
06JUN2013
07JUN2013
10JUN2013
11JUN2013
12JUN2013
13JUN2013
14JUN2013
17JUN2013
18JUN2013
19JUN2013
20JUN2013
21JUN2013
24JUN2013
25JUN2013
26JUN2013
27JUN2013
28JUN2013
01JUL2013
02JUL2013
03JUL2013
05JUL2013
08JUL2013
09JUL2013
10JUL2013
11JUL2013
12JUL2013
15JUL2013
16JUL2013
17JUL2013
18JUL2013
19JUL2013
22JUL2013
23JUL2013
24JUL2013
25JUL2013


KeyboardInterrupt: 

In [86]:
print tmp.shape
print covar.shape

(77L, 5L)
(5L, 5L)


In [98]:
high_moments['05MAR2013']['coskew']

array([[ 1.17342144e-09, -4.49266684e-10, -2.63970320e-09,
         7.06113190e-10, -7.69685897e-10, -4.49266684e-10,
         6.62260287e-11,  1.07885944e-09, -7.29128390e-10,
         3.97758403e-11, -2.63970320e-09,  1.07885944e-09,
         6.73904723e-09, -1.02917845e-09,  1.84633531e-09,
         7.06113190e-10, -7.29128390e-10, -1.02917845e-09,
         1.58577327e-09,  1.28157125e-09, -7.69685897e-10,
         3.97758403e-11,  1.84633531e-09,  1.28157125e-09,
         1.19129939e-09],
       [-4.49266684e-10,  6.62260287e-11,  1.07885944e-09,
        -7.29128390e-10,  3.97758403e-11,  6.62260287e-11,
        -4.58494550e-09,  1.95943010e-09, -4.31460731e-10,
        -7.73860613e-11,  1.07885944e-09,  1.95943010e-09,
        -4.50450754e-10,  2.23975826e-09, -2.54315099e-10,
        -7.29128390e-10, -4.31460731e-10,  2.23975826e-09,
        -1.30294175e-09,  2.17734675e-12,  3.97758403e-11,
        -7.73860613e-11, -2.54315099e-10,  2.17734675e-12,
         6.07295898e-10],
    

In [99]:
np.linalg.eigvals(high_moments['05MAR2013']['covar'])# positive definite

array([8.87920390e-07, 6.27050293e-06, 5.29326479e-06, 2.42364108e-06,
       2.64757762e-06])